In [ ]:
import pandas as pd
import numpy as np 
import keras
from keras_bert import get_base_dict, get_model, compile_model, gen_batch_inputs
from PyFiles import Functions as func
from PyFiles import Preprocessing as process

In [ ]:
ticker = 'SPY'

df, df_dict = process.combine_headlines(ticker)

In [ ]:
max_feat = 10000 
min_df = 20
max_df = 1.0 
ngram = (1,2)
pre_type = 'stem'
vect_type = 'tfidf'

new_df, x_train, x_test, y_train, y_test, preprocessing_dict = process.preprocess_tts(df, pre_type = pre_type, ngram = ngram, 
                                                                              max_features = max_feat, 
                                                                             min_df = min_df, max_df =max_df, 
                                                                              vect_type = vect_type)

In [ ]:
new_df.head(5)

In [ ]:
sentence_pairs = new_df.Headlines.map(lambda x: x.split(' ')).tolist()

[TUTORIAL](https://pypi.org/project/keras-bert/#Train-&-Use)

[BETTER TUTORIAL](https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/)

In [ ]:
# Build token dictionary
token_dict = get_base_dict()  # A dict that contains some special tokens
for pairs in sentence_pairs:
    for token in pairs[0] + pairs[1]:
        if token not in token_dict:
            token_dict[token] = len(token_dict)
token_list = list(token_dict.keys())  # Used for selecting a random word


# Build & train the model
model = get_model(
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
)
compile_model(model)
model.summary()

In [ ]:
x_train_new = np.array(x_train.values)
x_test_new = np.array(x_test.values)

In [ ]:
model.fit(x_train_new, y_train,
    steps_per_epoch=1000,
    epochs=100,
    validation_data=(x_test_new, y_test),
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    ],
)


# Use the trained model
inputs, output_layer = get_model(
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
    training=False,      # The input layers and output layer will be returned if `training` is `False`
    trainable=False,     # Whether the model is trainable. The default value is the same with `training`
    output_layer_num=4,)  # The number of layers whose outputs will be concatenated as a single output.
                         # Only available when `training` is `False`.